# Convolutional Neural Network on pixel neighborhoods

This notebook reads the pixel-neighborhood data written out by the Dataflow program of [1_explore.ipynb](./1_explore.ipynb) and trains a simple convnet model on Cloud ML Engine.


In [4]:
%pip install cloudml-hypertune

Note: you may need to restart the kernel to use updated packages.


In [5]:
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [6]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [7]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [8]:
%load_ext autoreload
%aimport ltgpred

In [9]:
import tensorflow as tf
print(tf.__version__)

1.13.1


## Train CNN model locally

In [5]:
!mkdir -p preproc/tfrecord
!gsutil cp gs://$BUCKET/lightning/preproc_0.02_32_2/tfrecord/*-00000-* preproc/tfrecord

Copying gs://cloud-training-demos-ml/lightning/preproc_0.02_32_2/tfrecord/eval-00000-of-00253...
Copying gs://cloud-training-demos-ml/lightning/preproc_0.02_32_2/tfrecord/train-00000-of-00522...
| [2 files][  5.4 GiB/  5.4 GiB]   77.5 MiB/s                                   
Operation completed over 2 objects/5.4 GiB.                                      


In [ ]:
%%bash
export PYTHONPATH=${PWD}/ltgpred/
OUTDIR=${PWD}/cnn_trained
DATADIR=${PWD}/preproc/tfrecord
rm -rf $OUTDIR
mkdir -p $OUTDIR
python3 -m trainer.train_cnn \
    --train_steps=10 --num_eval_records=512 --train_batch_size=16 --num_cores=1 --nlayers=5 --arch=convnet \
    --job-dir=$OUTDIR --train_data_path=${DATADIR}/train* --eval_data_path=${DATADIR}/eval*

In [34]:
!find /home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/

/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/1556043088
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/1556043088/variables
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/1556043088/variables/variables.data-00000-of-00001
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/1556043088/variables/checkpoint
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/1556043088/variables/variables.index
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/1556043088/assets
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/1556043088/assets/saved_model.json
/home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/155

In [35]:
%%bash
saved_model_cli show --all --dir $(ls -d -1 /home/jupyter/training-data-analyst/blogs/lightning/cnn_trained/export/exporter/* | tail -1)


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: init_1
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['ltg'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4225)
        name: ltg:0
    inputs['ref'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4225)
        name: ref:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['model'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: model/dense_8/Sigmoid:0
  Method name is: tensorflow/serving/predict


In [ ]:
%%bash
export CLOUDSDK_PYTHON=$(which python3)
OUTDIR=${PWD}/cnn_trained
DATADIR=${PWD}/preproc/tfrecord
rm -rf $OUTDIR
gcloud ml-engine local train \
    --module-name=trainer.train_cnn --package-path=${PWD}/ltgpred/trainer \
    -- \
    --train_steps=10 --num_eval_records=512 --train_batch_size=16 --num_cores=1 --nlayers=5 \
    --job-dir=$OUTDIR --train_data_path=${DATADIR}/train* --eval_data_path=${DATADIR}/eval*

## Training lighting prediction model on CMLE using GPU

custom_model_m_gpu is a machine with 4 K-80 GPUs.

In [ ]:
%%writefile largemachine.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: complex_model_m_p100

In [ ]:
%%bash

#DATADIR=gs://$BUCKET/lightning/preproc/tfrecord
DATADIR=gs://$BUCKET/lightning/preproc_0.02_32_2/tfrecord

#for ARCH in feateng convnet dnn resnet; do
for ARCH in convnet; do
  JOBNAME=ltgpred_${ARCH}_$(date -u +%y%m%d_%H%M%S)
  OUTDIR=gs://${BUCKET}/lightning/${ARCH}_trained_gpu
  gsutil -m rm -rf $OUTDIR
  gcloud ml-engine jobs submit training $JOBNAME \
      --module-name trainer.train_cnn --package-path ${PWD}/ltgpred/trainer --job-dir=$OUTDIR \
      --region=${REGION} --scale-tier CUSTOM --config largemachine.yaml \
      --python-version 3.5 --runtime-version 1.10 \
      -- \
      --train_data_path ${DATADIR}/train-* --eval_data_path ${DATADIR}/eval-* \
      --train_steps 5000 --train_batch_size 256 --num_cores 4 --arch $ARCH \
      --num_eval_records 1024000 --nlayers 5 --dprob 0 --ksize 3 --nfil 10 --learning_rate 0.01 
done


### Results (Dropout=0)
| Architecture | Training time | Validation RMSE | Validation Accuracy |
| --- | --- | --- | --- |
| feateng | 23 min | 0.2620 | 0.8233 |
| dnn | 62 min | 0.2752 | 0.8272 |
| convnet | 24 min | 0.2261 | 0.8462 |
| resnet | 63 min | 0.3088 | 0.7142 |

### Results (Dropout=0.05)
| Architecture | Training time | Validation RMSE | Validation Accuracy |
| --- | --- | --- | --- |
| feateng | 20 min | 0.2641 | 0.8258 |
| dnn | 58 min | 0.2284 | 0.8412 |
| convnet | 23 min | 0.2268 | 0.8459 |
| resnet | 80 min | 0.3005 | 0.6887 |


Other than for the dnn, dropout doesn't seem to help.  Based on these results, let's train a <b> convnet with no dropout </b>.

All the results above are for 
<pre>
--train_steps=5000 --train_batch_size=256 --num_cores=4 --arch=$ARCH \
--num_eval_records=1024000 --nlayers=5 --dprob=... --ksize=3 --nfil=10 --learning_rate=0.01
</pre>

### Hyperparameter tuning on GPU

In [45]:
%%writefile hyperparam_gpu.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: complex_model_m_p100
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 30
    maxParallelTrials: 2
    hyperparameterMetricTag: val_acc
    params:
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.01
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE
    - parameterName: nfil
      type: INTEGER
      minValue: 5
      maxValue: 30
      scaleType: UNIT_LINEAR_SCALE
    - parameterName: nlayers
      type: INTEGER
      minValue: 1
      maxValue: 5
      scaleType: UNIT_LINEAR_SCALE     
    - parameterName: train_batch_size # has to be multiple of 128
      type: DISCRETE
      discreteValues: [128, 256, 512, 1024, 2048, 4096]       

#    - parameterName: arch
#      type: CATEGORICAL
#      categoricalValues: ["convnet", "feateng", "resnet", "dnn"]       

Overwriting hyperparam_gpu.yaml


In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/lightning/convnet_trained_gpu_hparam
DATADIR=gs://$BUCKET/lightning/preproc_0.02_32_2/tfrecord
JOBNAME=ltgpred_hparam_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --module-name=ltgpred.trainer.train_cnn --package-path=${PWD}/ltgpred --job-dir=$OUTDIR \
    --region=${REGION} --scale-tier=CUSTOM --config=hyperparam_gpu.yaml \
    --python-version=3.5 --runtime-version=1.10 \
    -- \
    --train_data_path=${DATADIR}/train-* --eval_data_path=${DATADIR}/eval-* \
    --train_steps=5000 --train_batch_size=256 --num_cores=4 --arch=convnet \
    --num_eval_records=1024000 --nlayers=5 --dprob=0 --ksize=3 --nfil=10 --learning_rate=0.01 --skipexport

The hyperparameter training took 7.5 hours for me, cost 215 ML units (about 110USD list price) and had this as the best set of parameters:
<pre>
{
      "trialId": "2",
      "hyperparameters": {
        "nfil": "10",
        "learning_rate": "0.02735530997243607",
        "train_batch_size": "1024",
        "nlayers": "3"
      },
      "finalMetric": {
        "trainingStep": "1",
        "objectiveValue": 0.846787109375
      }
    },
</pre>

## Training lightning prediction model on CMLE using TPUs

Next, let's train on the TPU. Because our batch size is 4x, we can train for 4x fewer steps

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/lightning/cnn_trained_tpu
DATADIR=gs://$BUCKET/lightning/preproc_0.02_32_2/tfrecord
JOBNAME=ltgpred_cnn_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --module-name ltgpred.trainer.train_cnn --package-path ${PWD}/ltgpred --job-dir=$OUTDIR \
    --region ${REGION} --scale-tier BASIC_TPU \
    --python-version 3.5 --runtime-version 1.12 \
    -- \
    --train_data_path ${DATADIR}/train* --eval_data_path ${DATADIR}/eval* \
    --train_steps 1250 --train_batch_size 1024 --num_cores 8  --use_tpu \
    --num_eval_records 1024000 --nlayers 5 --dprob 0 --ksize 3 --nfil 10 --learning_rate 0.01

When I ran it, training finished with accuracy=0.82 (no change)

## More data, harder problem

In [ ]:
%%bash

#DATADIR=gs://$BUCKET/lightning/preproc/tfrecord
#DATADIR=gs://$BUCKET/lightning/preproc_0.02_32_2/tfrecord
DATADIR=gs://$BUCKET/lightning/preproc_0.02_32_1/tfrecord  # also 5-min validity

#for ARCH in feateng convnet dnn resnet; do
for ARCH in feateng; do
  JOBNAME=ltgpred_${ARCH}_$(date -u +%y%m%d_%H%M%S)
  OUTDIR=gs://${BUCKET}/lightning/${ARCH}_trained_gpu
  gsutil -m rm -rf $OUTDIR
  gcloud ml-engine jobs submit training $JOBNAME \
      --module-name ltgpred.trainer.train_cnn --package-path ${PWD}/ltgpred --job-dir=$OUTDIR \
      --region=${REGION} --scale-tier CUSTOM --config largemachine.yaml \
      --python-version 3.5 --runtime-version 1.12 \
      -- \
      --train_data_path ${DATADIR}/train-* --eval_data_path ${DATADIR}/eval-* \
      --train_steps 5000 --train_batch_size 256 --num_cores 4 --arch $ARCH \
      --num_eval_records 1024000 --nlayers 5 --dprob 0 --ksize 3 --nfil 10 --learning_rate 0.01 
done

With 100 days of data:

| Architecture | Training time | Validation RMSE | Validation Accuracy |
| --- | --- | --- | --- |
| feateng | 26 min | 0.2986 | 0.7901 |
| dnn | 40 min | 0.2995 | 0.7916 |
| convnet | 28 min | 0.2735 | 0.8141 |
| resnet | 68 min | 0.5135 | 0.4852 |

With full year of data:

| Architecture | Training time | Validation RMSE | Validation Accuracy |
| --- | --- | --- | --- |
| feateng | 26 min | 0.2986 | 0.7901 |
| dnn | 240 min | 0.2682 | 0.8114 |
| convnet | 150 min | 0.2557 | 0.8192 |
| resnet | 68 min | 0.5135 | 0.4852 |


Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License